In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import gradient_descent_v2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

def eachFile(filepath):
	pathDir = os.listdir(filepath)
	out = []
	for allDir in pathDir:
		child = allDir
		out.append(child)
	return out


NUM_CLASSES = 11
TRAIN_PATH = 'dataset/train/'
TEST_PATH = 'dataset/test/'

FC_NUMS = 4096

FREEZE_LAYERS = 17

IMAGE_SIZE = 224

base_model = VGG19(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True

model.compile(optimizer=gradient_descent_v2.SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(directory=TRAIN_PATH,
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), classes=eachFile(TRAIN_PATH))
filepath = 'model/sewer_weight.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

history_ft = model.fit_generator(train_generator, epochs=10, callbacks=[checkpoint])
model.save('model/sewer_weight_final.h5')

In [6]:

   
from tensorflow.python.keras.initializers import glorot_uniform
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
from keras.utils.generic_utils import CustomObjectScope
from keras.preprocessing.image import  img_to_array

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

# image folder
folder_path = 'dataset/test/'

# path to model
model_path = 'model/sewer_weight.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

# dimensions of images
img_width, img_height = 224, 224


i = 0
images = []
for img in os.listdir(folder_path):
   img1 = image.load_img(os.path.join(folder_path, img), target_size=(img_width, img_height))
   img2 = img_to_array(img1)
   img2 = np.expand_dims(img2, axis=0)
   classes = model.predict(img2)[0]
   idxs = np.argsort(classes)[::-1][:1]

   classname = ['UD_IN', 'UD_PJ', 'JF', 'BK', 'JD', 'SD', 'DS', 'ETC',
                'CC', 'CL', 'LP']

   out = cv2.imread(os.path.join(folder_path, img))

   for (i, j) in enumerate(idxs):
       label = "{}:{:.2f}%".format(classname[idxs[i]], classes[idxs[i]] * 100)
       cv2.putText(out, label, (10, (i * 30) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

   cv2.imwrite("visualization/sd/%s"%img,out)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x17f588860>

In [ ]:
import os
import cv2
bad_list=[]
dir=r'dataset/train/'
subdir_list=os.listdir(dir) # create a list of the sub directories in the directory ie train or test
for d in subdir_list:  # iterate through the sub directories train and test
    dpath=os.path.join (dir, d) # create path to sub directory
    if d in ['test', 'train']:
        class_list=os.listdir(dpath) # list of classes ie dog or cat
       # print (class_list)
        for klass in class_list: # iterate through the two classes
            class_path=os.path.join(dpath, klass) # path to class directory
            #print(class_path)
            file_list=os.listdir(class_path) # create list of files in class directory
            for f in file_list: # iterate through the files
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') # find index of period infilename
                ext=f[index+1:] # get the files extension
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)